<a href="https://colab.research.google.com/github/makler322/msu_ml_spring_2021/blob/master/MTS_summer_school/IiIsNotMine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Рекомендательная система**

Имеются следующие базы: Книг (Автор, Название, Год, Идентификатор, Жанр)
                        Пользователей (Идентификатор, Возраст, Пол)
                        Взаимодействия (Идентификатор пользователя, Идентификатор книги, Прогресс прочтения книги пользователем в процентах
                                            Оценка пользователя, Дата открытия книги)

Цель: Наиболее точно порекомендовать список из 10 книг для каждого пользователя из тестовой выборки.
Так как мне не хотят сообщать что нужно вычислить факт прочтения или факт открытия (как сообщать дострою еще варианты, блокнот получится длиииным)))
                     буду пока оценивать по своей метрике.

1. Загрузка, обработка и стыковка данных
2. модель **Tensor Flow** раотающая на **TPU**   (в 16 раз ускорило работу сети) модель с четырмя раздельными входами: 
          1 - принимает id пользователя, разбивая на вектор эмбединга
          2 - принимает id книги, разбивая на вектор эмбединга
          3 - принимает массив числовые характеристик книг и пользователя и их взаимодействий
          4 - принимает вектор описания жанра объединенное с автором разбитый на токены
          На выход подается id книги с которым пользователь взаимодействовал после книги поданной на вход

Конечно времени не так много и проект тестовый и реальную метрику не уточняют мне в задании, но рабочий можно было бы еще оптимизировать.
Оптимизировать код весь для работы в компании где маленький процент точности дает результаты хорошие в бизнесе, расчиеты, вычисления в матричные по возмможности все перевести, после этого и  с сетью проще будет и быстрее работать. Далее я бы использовал парсер сайтов чтоб собрать описания книг и возможно комментарии к ним объеденив комменатрии к каждой книге тоже в эмбединг пропустив через lstm сеть, используя Bert к примеру, количество страниц и т.д. Наши данные здесь и модель далеки от совершенства, но ядро думаю очень неплохо составленно. Попробовать первый слой для входа описаний использовать так же Conv слои из компьютерного зрения собирающие признаки в ядра свертки. С LSTM бы сняли больше смысловой инфы с текстов, сверточные слои в качестве енкодеров после них ускорили бы значительно работу сети. Написать Функцию кросс валидации, потому что для этой сети с TPU сырая библиотека и нет готовой, но думаю проблемы ни для кого не составит. Организовать систему подбора гипер параметров. это уже легкие задачи, но перед этим нужно значительно оптимизировать все.

Времени мало. Для идеального решения такой задачи важно так же сравнить это все с работой других моделей и алгоритмов машинного обучения, Пробовать стекинг объязательно, я пробовал на другой задаче стекинг нейросетей с простыми алгоритмами и бустингами. Это все очень занимательно и важно для задачи и хочется все модели построить проверить. Впрочем может позже будет поставлена такая задача, где я смогу собрать это все провести детальный анализ и написать статью хорошо оформленную, где распишу каждый шаг. думаю этого в сети не хватает)))

In [11]:
#!pip install ml_metrics
!pip install lightfm

     |████████████████████████████████| 317kB 3.9MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705351 sha256=88967e689e1ba9fc1ca8e3c378db1e1a6f9ea9a1ea35e90d2bdad9352781a2d8
  Stored in directory: /root/.cache/pip/wheels/c6/64/d4/673c7277f71ac4c5ad4835b94708c01b653ef2d3aa78ef20aa
Successfully built lightfm


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# Импорт библиотек
import warnings
warnings.filterwarnings("ignore")

import os
import re
import sys
import nltk
import time
import pickle
import string
import datetime
import ml_metrics 
import numpy as np
import pandas as pd
from  more_itertools import unique_everseen
import tensorflow as tf
import plotly.graph_objs as go
import plotly.figure_factory as ff

from tqdm.auto import tqdm
from lightfm.data import Dataset
from collections import deque
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import coo_matrix
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from scipy.sparse import vstack
from scipy import sparse
from lightfm import LightFM
from sklearn.metrics.pairwise import cosine_similarity

from scipy import stats
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from subprocess import check_output
from scipy.special import boxcox1p

from tensorflow.keras.layers import Input, Embedding, Reshape, Dot, Concatenate, Dense, Dropout, BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam, Adagrad, SGD
from tensorflow.python.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')
stop = stopwords.words('russian')
stop_words_ = set(stopwords.words('russian'))
wn = WordNetLemmatizer()

tf.__version__

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


'2.4.1'

Запуск обнаружения наличия GPU и вывод количества потоков доступных для распаралеливания вычислений. Не забудьте выключить GPU в блокноте

In [14]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))
strategy = tf.distribute.MirroredStrategy(devices = ["GPU:0", "GPU:1"])

GPU device not found
Found GPU at: 
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


Запуск обнаружения наличия TPU и вывод количества потоков доступных для распаралеливания вычислений. Не забудьте выключить TPU в блокноте

In [15]:
'''try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print('Running on TPU ', tpu.master())
except ValueError:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)'''

'try:\n    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()\n    tf.config.experimental_connect_to_cluster(tpu)\n    tf.tpu.experimental.initialize_tpu_system(tpu)\n    strategy = tf.distribute.experimental.TPUStrategy(tpu)\n    print(\'Running on TPU \', tpu.master())\nexcept ValueError:\n    strategy = tf.distribute.get_strategy()\n\nprint("REPLICAS: ", strategy.num_replicas_in_sync)'

Ограничение вывода числа знаков после запятой до трех в таблицах, наших и просмотр фаайлов в каталоге


In [16]:
#pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))
#print(check_output(["ls", "../input/mts-ml-summer-school"]).decode("utf8"))

Загрузка данных

In [17]:
submission = pd.read_csv('/content/drive/MyDriv/MTS_summer_school/sample_submission.csv')
users = pd.read_csv('/conteent/drive/MyDrive/MTS_summer_school/users.csv')
items = pd.read_csv('/content/drive/MyDrive/MTS_summer_school/items.csv')
interactions = pd.read_csv('/content/drive/MyDrive/MTS_summer_school/interactions.csv')

Функция проверки отсутствующих пользователей из целевого списка в будущем обучающем датафрейме. Проверил и с другими таблицами, о двух пользователях выведенных в целевой список нет вообще никакакой информации, но их важно добавить, иначе сеть выдаст ошибку когда примет их в прогнозирование, т.к. их ид не найдется место в эмбединге

In [18]:
def miss_user(table):
    return len(list(set(submission.Id.unique()) - set(table.user_id.unique())))
miss_user(interactions)

2

Вывод размерности таблиц данных и выведем информацию о таблице пользователей, с него и начнем.

In [19]:
# Вывод размерности таблиц данных
print('users shape - ', users.shape, '\nitems shape - ', items.shape, '\ninteractions shape - ', interactions.shape)
# Просмотр уникальных значений возраста таблицы пользователей
print('We have {} unique values in {} column in users age: {}'.format(len(users['age'].unique()), 'age', users['age'].unique()))
# Вывод информации о пользователях
users.info()

users shape -  (137254, 3) 
items shape -  (63758, 5) 
interactions shape -  (1562617, 5)
We have 7 unique values in age column in users age: ['45_54' '25_34' '65_inf' '18_24' '35_44' '55_64' nan]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137254 entries, 0 to 137253
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  137254 non-null  int64  
 1   age      137244 non-null  object 
 2   sex      135640 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.1+ MB


Смотрим распределение признака возраста

In [20]:
group_age_m = users.loc[users['sex'] == 1].groupby('age').count()  
group_age_f = users.loc[users['sex'] == 0].groupby('age').count()
fig = go.Figure(data = [go.Bar(x = group_age_m.index.values, y = group_age_m['user_id'], text = 'men', textposition = 'outside', textfont_color = "red"), 
                        go.Bar(x = group_age_f.index.values, y = group_age_f['user_id'], text = 'women', textposition = 'outside', textfont_color = "red")])

fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6, name = '')
fig.update_layout(title_text='Распределение признака возвраста', width = 1000, height = 600)
fig.show()

Функция просмотра пустых значений признаков в процентах

In [21]:
def miss(data):
    miss_data = data.copy(deep = True)
    miss_data_na = (miss_data.isnull().sum() / len(miss_data)) * 100
    miss_data_na = miss_data_na.drop(miss_data_na[miss_data_na == 0].index).sort_values(ascending = False)[:30]
    missing_data = pd.DataFrame({'Missing Ratio' :miss_data_na})
    missing_data
    return(missing_data)

miss(users).head(5)

,Missing Ratio
sex,1.175922
age,0.007286


Смотрим распределение отсутвующих значений

In [22]:
# Смотрим распределение отсутвующих значений
def miss_dist(data, group, feature):
    data = data.copy(deep = True)
    data['null'] = data[feature].isnull()
    data.loc[data['null'] == True].groupby(group).count()
    return data

miss_dist(users, 'age', 'sex').head(5)

,user_id,age,sex,null
0,0,45_54,1.0,False
1,1,25_34,0.0,False
2,2,45_54,0.0,False
3,3,65_inf,0.0,False
4,4,18_24,0.0,False


Удаляем отсутствующие значения

In [23]:
users_mod = users.dropna(inplace = False)
print(users.shape, '  /  ', users_mod.shape)

(137254, 3)   /   (135630, 3)


Выведем следующую таблицу

In [24]:
print(items.info())
# Просмотр уникальных значений возраста
print('We have {} unique values in {} column in users authors: {}'.format(len(items['authors'].unique()), 'authors', items['authors'].unique()))
items.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63758 entries, 0 to 63757
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       63758 non-null  int64 
 1   title    63758 non-null  object
 2   genres   63753 non-null  object
 3   authors  56700 non-null  object
 4   year     49508 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.4+ MB
None
We have 17025 unique values in authors column in users authors: ['Михаил Салтыков-Щедрин' 'Антон Чехов' 'Михаил Лермонтов' ...
 'Евлампий Бесподобный' 'Екатерина Балобанова' 'Микаель Ниеми']


,id,title,genres,authors,year
0,248031,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,256084,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,134166,Испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869
3,281311,Странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831
4,213473,Господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1873


Сказать я удилен это как промолчать, более 11т. жанров))) потом глянем)) страшно сразу об этом думать и исследовать. На этом этапе я и решил делать эмбединги. Выведем пока пустые значения признаков в процентах

In [25]:
miss(items).head(5)

,Missing Ratio
year,22.350136
authors,11.069983
genres,0.007842


Здесь поинтереснее)) Попробуем найти дубликаты по названию и автору чтобы восстановить часть данных по годам. Дубликаты удалять не будем из за связи с другими таблицами, можно их потом в конечном датасете конечно склеить, но здесь не так важно, их слишком мало по сравнению со всей таблицей, был бы высокий процент дубликатов это возможно повлияло бы на переобучение, ну позже глянем и дизбаланс классов

In [26]:
items_dubl = items.copy(deep = True)
items_dubl = items_dubl[items_dubl.duplicated(subset = ['title', 'authors'])]
items_dubl['null'] = items_dubl['year'].isnull()
print('Количество дубликатов записей с разными id  -  ', items_dubl.shape[0])
print(items[items.duplicated(subset = ['title', 'authors'])].loc[items['title'] == 'Невеста'])
items_dubl.head(3)

Количество дубликатов записей с разными id  -   3922
           id    title  ...      authors  year
632    129327  Невеста  ...  Антон Чехов  1903
40780  113597  Невеста  ...  Антон Чехов  1903
40781  199636  Невеста  ...  Антон Чехов  1903
50363  104536  Невеста  ...  Антон Чехов   NaN

[4 rows x 5 columns]


,id,title,genres,authors,year,null
82,258306,Басни,"Стихи и поэзия,Литература 19 века,Русская клас...",Иван Крылов,NaN,True
152,300964,Басни,"Стихи и поэзия,Литература 19 века,Русская клас...",Иван Крылов,NaN,True
632,129327,Невеста,"Классическая проза,Русская классика",Антон Чехов,1903,False


Ищем в двух столбцах одновременно пропущенные признаки

In [27]:
items_dubl_none = items_dubl.loc[items_dubl['null'] == True]
items_dubl_nonone = items_dubl.loc[items_dubl['null'] == False]
print('Количество дубликатов записей с разными id  -  ', items_dubl_none.shape[0])
items_dubl_nonone.head(3)

Количество дубликатов записей с разными id  -   979


,id,title,genres,authors,year,null
632,129327,Невеста,"Классическая проза,Русская классика",Антон Чехов,1903,False
1659,188635,Белолобый,"Зарубежная классика,Литература 19 века,Русская...",Антон Чехов,1895,False
1956,95545,Сказка о рыбаке и рыбке,"Детские стихи,Сказки",Александр Пушкин,1833,False


Уже не так интересно)) скоро получится что это все ради исправления одной десятой процента данных))) Ну продолжим раз начали

In [28]:
items_dubl_join = items_dubl_nonone.merge(items_dubl_none, on = ['title', 'authors']).drop_duplicates('id_y')
items_dubl_join = items_dubl_join[['id_y', 'year_x']]
items_dubl_join.rename({'id_y': 'id'}, axis = 1, inplace=True)
items_dubl_join['id'] = items_dubl_join['id'].astype('int64')
items_re = items.copy(deep = True)
items_re = items_re.merge(items_dubl_join, how = 'left')
items_re['null'] = items_re['year'].isnull()
items_re.loc[items_re['null'] == True, 'year'] = items_re.loc[items_re['null'] == True, 'year_x']
items_re = items_re.drop(['year_x', 'null'], axis=1)
print('Количество востановленных данных  -  ', items_dubl_join.shape[0])
items_re.head(3)

Количество востановленных данных  -   104


,id,title,genres,authors,year
0,248031,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,256084,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,134166,Испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869


Ищем пропущенные года с известными авторами

In [29]:
items_null = items_re.copy(deep = True)
items_null = items_null[items_null['year'].isnull() & items_null['authors'].notnull()]
print('Количество пропущенных годов с указаными авторами  -  ', items_null.shape[0])

Количество пропущенных годов с указаными авторами  -   13474


Уже лучше)) почти все года можно восстановить, потом на модели потестим помогло ли нам это все))

Просмотр уникальных значений авторов в строках с пропущенными годами

In [30]:
print('We have {} unique values in {} column in users authors: {}'.format(len(items_null['authors'].unique()), 'authors', items_null['authors'].unique()))

We have 5871 unique values in authors column in users authors: ['Михаил Салтыков-Щедрин' 'Ги де Мопассан' 'Николай Гоголь' ...
 'Радмила Хакова' 'Лана Мейер' 'Сергей Фонвизин']



Ого))) ничего интересного)) ну мб это все окупится парой процентов хоть улучшения модели. продолжим, не торопимся ведь пока. Вынесем пока их в список

In [31]:
authors_null_year = items_null['authors'].unique().tolist()
authors_null_year[:3]

['Михаил Салтыков-Щедрин', 'Ги де Мопассан', 'Николай Гоголь']

Приводим даты к общему типу замменяя текст на числа, парсим форматы отличные от %Y на части, берем первые 4 цифры, последние 4 выносим в отдельную колонку

Потом заменяем обрезанные форматы, числа меньше 70ты указал убрать месяца, заменяем на значения другой колонки где число года (они то там то там))

In [32]:
items_re_year = items_re.copy(deep = True)
items_re_year['year2'] = items_re_year['year'].astype(str).str[-4:]
items_re_year['year'] = items_re_year['year'].astype(str).str[:4]
list1 = ['-', '–', ',', ' ', '.', '_', ':']
list2 = ['Октя', 'Нояб', 'до', 'окол', 'ок', 'перв', '. э.', '', 'алим', 'н', 'dtrf', 'век', '019й']
dict1 = {'XII в.': 1150, 'IX.': 850,'II в. н. э.': 150,'II': 150,'I': 50,'III': 250,'425 до н. э.': -425,
        '407 до н. э.': -407, '420 до н. э.': -420,'XII ': 1150,'III—': 300,'IV': 350,'XII': 650, 'IX': 850,
        'XII': 650, 'IX': 850,'I': 150}

def year_mod(data, feature, list1, list2, dict1):
    for i in list1:
        data = data.assign(year = data.year.str.split(i).str[0])
        data = data.assign(year2 = data.year2.str.split(i).str[0])
    for i in list2:
        data.loc[:,feature] = data.loc[:,feature].replace(i,np.NaN)
    for i in dict1:
        data.loc[data.loc[data[feature] == i][feature].index, feature] = dict1[i]
    return data

items_re_year = year_mod(items_re_year, 'year', list1, list2, dict1)
items_re_year = year_mod(items_re_year, 'year2', list1, list2, dict1)
items_re_year.loc[items_re_year.loc[items_re_year.loc[:,'year'] == 'nan']['year'].index, 'year'] = '5000'
items_re_year.loc[items_re_year.loc[items_re_year.loc[:,'year2'] == 'nan']['year2'].index, 'year2'] = '5000'
items_re_year.loc[:,'year'] = items_re_year.loc[:,'year'].fillna(5000)
items_re_year.loc[:,'year'] = items_re_year.loc[:,'year'].values.astype(int)
items_re_year.loc[:,'year2'] = items_re_year.loc[:,'year2'].fillna(5000)
items_re_year.loc[:,'year2'] = items_re_year.loc[:,'year2'].values.astype(int)
items_re_year.year[(items_re_year.year == 5000) & (items_re_year.year2 > 70)] = items_re_year.year2[(items_re_year.year == 5000) & (items_re_year.year2 > 70)]
items_re_year.year2[(items_re_year.year2 == 5000) & (items_re_year.year > 70)] = items_re_year.year[(items_re_year.year2 == 5000) & (items_re_year.year > 70)]
items_re_year.year[(items_re_year.year < 70) & ((items_re_year.year2 < 5000) & (items_re_year.year2 > 70))] = items_re_year.year2[
                                    (items_re_year.year < 70) & ((items_re_year.year2 < 5000) & (items_re_year.year2 > 70))]
items_re_year.year2[(items_re_year.year2 < 70) & ((items_re_year.year < 5000) & (items_re_year.year > 70))] = items_re_year.year[
                                    (items_re_year.year2 < 70) & ((items_re_year.year < 5000) & (items_re_year.year > 70))]

Выносим id строк которые неправильно возможно обрезались, чтоб посмотреть их значения в начальном датафрейме

In [33]:
break_data = pd.merge(items_re[['id','year']], items_re_year[(items_re_year.year < 70) | (items_re_year.year2 < 70)][['id']], on = ['id', 'id'])
print('Количество испорченных дат  -  ', break_data.shape[0])
break_data.head(3)

Количество испорченных дат  -   26


,id,year
0,9508,58-50 гг. до н. э.
1,23030,Октябрь 2000 – апрель 2001 г.
2,23065,до 1885 г.


Некоторые даты совсем в необычном формате, исправим их уже вручную

In [34]:
dict2 = {9508: -50, 23030: 2001, 23065: 1885, 331107: 1755, 301204: 1837, 195963: 1832, 206005: 1900, 232479: 1849, 201027: 1900, 
        190958: 1870, 93468: 1900, 54667: 1968, 145079: 1878, 141866: 1849, 350748: 1937, 92583: 1972, 327040: 1932,
        96045: 1940, 345023: 2018, 135516: -50, 289045: 1836, 139843: 1972, 341987: 1935, 286331: 1930, 233918: 1942, 72151: 1850}
for i in dict2:
    items_re_year.loc[items_re_year.loc[items_re_year['id'] == i]['id'].index, 'year'] = dict2[i]
    items_re_year.loc[items_re_year.loc[items_re_year['id'] == i]['id'].index, 'year2'] = dict2[i]
items_re_year.year = ((items_re_year.year + items_re_year.year2) / 2)
items_re_year[30:50]
items_re_year = items_re_year.drop(['year2'], axis = 1)
items_re_year.year = items_re_year.year.astype(int)
print(items_re_year.loc[(items_re_year.year > 2030) & (items_re_year.year < 5000)])
items_re_year = items_re_year.loc[items_re_year.id != 372466]
items_re_year.head(3)

           id                 title  ...        authors  year
49129  372466  Заманчивое наказание  ...  Кэтти Уильямс  2917

[1 rows x 5 columns]


,id,title,genres,authors,year
0,248031,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,256084,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,134166,Испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869


Теперь можно немного поплакать, прежду чем продолжить))) и попробуем восстановить еще пропущенные значения заполняя года средними значениями из списка книг авторов

Создаем таблицу авторов и периода активности))) не забываем что пропущенные значения закодированны большим значением, временно

In [35]:
items_year_recover = items_re_year.copy(deep = True)
authors_list = items_re_year.loc[items_re_year.year != 5000].groupby('authors').mean()
authors_list['authors'] = authors_list.index
authors_list = authors_list.drop(['id'], axis = 1).reset_index(drop = True)
authors_list.head(3)

,year,authors
0,2017.250000,Редакция журнала National Geographic
1,2018.666667,Редакция журнала Publish / Паблиш
2,2017.128205,Редакция журнала Сабрина


Заменяем пустые значения годов с подписанными авторами

In [36]:
items_year_recover['year_rec'] = items_year_recover.merge(authors_list, how = 'left', on='authors')['year_y']
items_year_recover.loc[items_year_recover.year == 5000, 'year'] = items_year_recover.loc[items_year_recover.year == 5000, 'year_rec']
items_year_recover.year = items_year_recover.year.fillna(5000).astype(int)
items_year_recover = items_year_recover.drop(['year_rec'], axis = 1)
print('Количество оставшихся пустых значений  -  ', items_year_recover.loc[items_year_recover.year == 5000].shape[0])
items_year_recover = items_year_recover.rename({'id': 'item_id'}, axis=1)
items_year_recover.head(3)

Количество оставшихся пустых значений  -   7831


,item_id,title,genres,authors,year
0,248031,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,256084,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,134166,Испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869


Смотрим пустыe значения в новом датафрейме

In [37]:
items_year_recover['year'] = items_year_recover['year'].replace(5000, np.NaN)
miss(items_year_recover).head(3)

,Missing Ratio
year,12.282573
authors,11.070157
genres,0.007842


Удалим данные без описания жанра и смотрим наличие различных книг с одинаковыми именами

In [38]:
items_year_recover.dropna(subset = ['genres'], inplace=True)
title_count = items_year_recover.groupby('title')['item_id'].agg(list)
title_count = title_count.map(len)
print(title_count.value_counts())
title_count[title_count > 9].head(5)

1     53767
2      3580
3       480
4       143
5        57
6        25
7        15
8         9
9         4
18        1
10        1
11        1
12        1
35        1
13        1
15        1
51        1
Name: item_id, dtype: int64


title
Возвращение                      18
Исповедь                         10
Полное собрание стихотворений    13
Противостояние                   11
Рассказы                         15
Name: item_id, dtype: int64

Глянем пример

In [39]:
items_year_recover.loc[items_year_recover.title == 'Возвращение'].head(5)

,item_id,title,genres,authors,year
4260,98549,Возвращение,"Литература 20 века,Русская классика",Андрей Платонов,1946.0
12004,79870,Возвращение,"Героическое фэнтези,Русское фэнтези",Александр Прозоров,2007.0
12735,34953,Возвращение,"Героическое фэнтези,Попаданцы,Юмористическое ф...",Юлия Фирсанова,2011.0
17430,157205,Возвращение,"Попаданцы,Боевое фэнтези",Юрий Иванович,2014.0
19587,147753,Возвращение,"Героическая фантастика,Боевое фэнтези",Николай Басов,2000.0


Теперь откроем таблицу взаимодействий глянем инфу и пустые значения

In [40]:
print(interactions.info())
print(interactions.shape)
print(miss(interactions).head(5))
interactions.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562617 entries, 0 to 1562616
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   user_id     1562617 non-null  int64  
 1   item_id     1562617 non-null  int64  
 2   progress    1562617 non-null  int64  
 3   rating      323571 non-null   float64
 4   start_date  1562617 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 59.6+ MB
None
(1562617, 5)
        Missing Ratio
rating      79.293007


,user_id,item_id,progress,rating,start_date
0,90133,82910,100,NaN,2018-01-01
1,159130,331068,70,NaN,2018-01-01
2,80061,26540,69,4.0,2018-01-01


Построим распределение признака прогресса

In [41]:
bar_progress = interactions.groupby('progress').count() 
fig = go.Figure(data = [go.Bar(x = bar_progress.index.values, y = bar_progress['user_id'])])

fig.update_traces(marker_color = 'rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6, name = '')
fig.update_layout(title_text = 'Распределение признака возвраста', width = 1000, height = 600)
fig.show()

Удалим одно наблюдение выходящее за пределы логики даже)))

In [42]:
interactions_mod = interactions.copy(deep = True)
interactions_mod = interactions_mod.loc[interactions_mod.progress <= 100]

Собираем общий датафрейм

In [43]:
all_data = interactions_mod.copy(deep = True)
all_data = all_data.merge(users_mod, how='left')
all_data = all_data.merge(items_year_recover, how='left')
all_data.head(3)

,user_id,item_id,progress,rating,start_date,age,sex,title,genres,authors,year
0,90133,82910,100,NaN,2018-01-01,55_64,0.0,Ведьма отмщения,Современные детективы,Галина Романова,2013.0
1,159130,331068,70,NaN,2018-01-01,35_44,0.0,Проклятый горн,"Героическое фэнтези,Боевое фэнтези",Алексей Пехов,2014.0
2,80061,26540,69,4.0,2018-01-01,25_34,0.0,Время – убийца,"Зарубежные детективы,Современные детективы",Мишель Бюсси,2016.0


Смотрим пропущенные значения и количество отсутствующих пользователей из целевого признака

In [44]:
print(list(set(submission.Id.unique()) - set(all_data.user_id.unique())))
miss(all_data).head(5)

[156235, 10309]


,Missing Ratio
rating,79.293057
sex,15.865510
age,15.865510
year,7.666631
authors,6.680144


Добовим их и выведем отношение пропущенных рейтиногов без авторов к тем что с авторами))

In [45]:
all_data = pd.concat([all_data, users.loc[(users.user_id == 10309) | (users.user_id == 156235)]], join = "outer")
print(all_data.loc[all_data.authors.isnull() & all_data.rating.notnull()].shape[0] / all_data.loc[all_data.authors.notnull() & all_data.rating.notnull()].shape[0])
all_data.shape

0.06208313638989549


(1562618, 11)

Процент конечно не большой, но попробуем сохранить данные указав что нет автора и закодировав в отдельный столбец

In [46]:
all_data.authors = all_data.authors.fillna('noauthors')
all_data['authors_code'] = all_data.authors
all_data.loc[(all_data['authors_code'] != 'noauthors'), 'authors_code'] = 1
all_data.loc[(all_data['authors_code'] == 'noauthors'), 'authors_code'] = 0
print(all_data.shape)
all_data.head(3)

(1562618, 12)


,user_id,item_id,progress,rating,start_date,age,sex,title,genres,authors,year,authors_code
0,90133,82910.0,100.0,NaN,2018-01-01,55_64,0.0,Ведьма отмщения,Современные детективы,Галина Романова,2013.0,1
1,159130,331068.0,70.0,NaN,2018-01-01,35_44,0.0,Проклятый горн,"Героическое фэнтези,Боевое фэнтези",Алексей Пехов,2014.0,1
2,80061,26540.0,69.0,4.0,2018-01-01,25_34,0.0,Время – убийца,"Зарубежные детективы,Современные детективы",Мишель Бюсси,2016.0,1


Сделаем наверно то же самое и с возрастом и полом там то процент выше))) ну почти тоже самое, это ведь совсем категориальные признаки

In [47]:
all_data.age = all_data.age.fillna('hz')
all_data.sex = all_data.sex.fillna('lesgey')
print(all_data.shape)
all_data.head(3)

(1562618, 12)


,user_id,item_id,progress,rating,start_date,age,sex,title,genres,authors,year,authors_code
0,90133,82910.0,100.0,NaN,2018-01-01,55_64,0,Ведьма отмщения,Современные детективы,Галина Романова,2013.0,1
1,159130,331068.0,70.0,NaN,2018-01-01,35_44,0,Проклятый горн,"Героическое фэнтези,Боевое фэнтези",Алексей Пехов,2014.0,1
2,80061,26540.0,69.0,4.0,2018-01-01,25_34,0,Время – убийца,"Зарубежные детективы,Современные детективы",Мишель Бюсси,2016.0,1


Давай закодируем года тоже по периодам под One hot encoding

In [48]:
all_data.year = all_data.year.fillna(5000).astype(int)
all_data.loc[(all_data['year'] < 2015) & (all_data['year'] >= 2012), 'year'] = 2015
all_data.loc[(all_data['year'] < 2012) & (all_data['year'] >= 2009), 'year'] = 2012
all_data.loc[(all_data['year'] < 2009) & (all_data['year'] >= 2005), 'year'] = 2012
all_data.loc[(all_data['year'] < 2005) & (all_data['year'] >= 2000), 'year'] = 2005
all_data.loc[(all_data['year'] < 2000) & (all_data['year'] >= 1995), 'year'] = 2000
all_data.loc[(all_data['year'] < 1995) & (all_data['year'] >= 1990), 'year'] = 1995
all_data.loc[(all_data['year'] < 1990) & (all_data['year'] >= 1980), 'year'] = 1990
all_data.loc[(all_data['year'] < 1980) & (all_data['year'] >= 1970), 'year'] = 1980
all_data.loc[(all_data['year'] < 1970) & (all_data['year'] >= 1965), 'year'] = 1970
all_data.loc[(all_data['year'] < 1965) & (all_data['year'] >= 1950), 'year'] = 1965
all_data.loc[(all_data['year'] < 1950) & (all_data['year'] >= 1930), 'year'] = 1950
all_data.loc[(all_data['year'] < 1930) & (all_data['year'] >= 1900), 'year'] = 1930
all_data.loc[(all_data['year'] < 1900) & (all_data['year'] >= 1850), 'year'] = 1900
all_data.loc[(all_data['year'] < 1850) & (all_data['year'] >= 1800), 'year'] = 1850
all_data.loc[(all_data['year'] < 1800) & (all_data['year'] >= 1700), 'year'] = 1800
all_data.loc[(all_data['year'] < 1700) & (all_data['year'] >= 1500), 'year'] = 1700
all_data.loc[(all_data['year'] < 1500) & (all_data['year'] >= 1200), 'year'] = 1500
all_data.loc[(all_data['year'] < 1200) & (all_data['year'] >= 700), 'year'] = 1200
all_data.loc[(all_data['year'] < 700), 'year'] = 700
all_data.year = all_data.year.astype(str)
print(all_data.shape)
all_data.head(3)

(1562618, 12)


,user_id,item_id,progress,rating,start_date,age,sex,title,genres,authors,year,authors_code
0,90133,82910.0,100.0,NaN,2018-01-01,55_64,0,Ведьма отмщения,Современные детективы,Галина Романова,2015,1
1,159130,331068.0,70.0,NaN,2018-01-01,35_44,0,Проклятый горн,"Героическое фэнтези,Боевое фэнтези",Алексей Пехов,2015,1
2,80061,26540.0,69.0,4.0,2018-01-01,25_34,0,Время – убийца,"Зарубежные детективы,Современные детективы",Мишель Бюсси,2016,1


Названия книг уберем)) они повторяются и нелогичны и их слишком много для кассификации и больше будут запутывтаь наши модели думаю. И преобразуем формат даты

In [49]:
all_data['start_date'] = pd.to_datetime(all_data['start_date'], format = '%Y-%m-%d')
print(all_data.shape)
all_data.head(3)

(1562618, 12)


,user_id,item_id,progress,rating,start_date,age,sex,title,genres,authors,year,authors_code
0,90133,82910.0,100.0,NaN,2018-01-01,55_64,0,Ведьма отмщения,Современные детективы,Галина Романова,2015,1
1,159130,331068.0,70.0,NaN,2018-01-01,35_44,0,Проклятый горн,"Героическое фэнтези,Боевое фэнтези",Алексей Пехов,2015,1
2,80061,26540.0,69.0,4.0,2018-01-01,25_34,0,Время – убийца,"Зарубежные детективы,Современные детективы",Мишель Бюсси,2016,1


Ну и на десерт остается у нас парсинг и объеденение жанра с автором, под эмбединг пойдет. и с названии автара уберем пробелы.

In [50]:
all_data['authors'] = all_data['authors'].replace(r'\s+','',regex = True)
print(all_data.shape)
all_data.head(3)

(1562618, 12)


,user_id,item_id,progress,rating,start_date,age,sex,title,genres,authors,year,authors_code
0,90133,82910.0,100.0,NaN,2018-01-01,55_64,0,Ведьма отмщения,Современные детективы,ГалинаРоманова,2015,1
1,159130,331068.0,70.0,NaN,2018-01-01,35_44,0,Проклятый горн,"Героическое фэнтези,Боевое фэнтези",АлексейПехов,2015,1
2,80061,26540.0,69.0,4.0,2018-01-01,25_34,0,Время – убийца,"Зарубежные детективы,Современные детективы",МишельБюсси,2016,1


Займемся объединением и чисткой текста) Определим пока функции

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token) > 2 
def clean_txt(text):
    clean_text = []
    clean_text2 = []
    text = re.sub("'", "",text)
    text=re.sub("(\\d|\\W)+"," ",text) 
    text = text.replace("nbsp", "")
    clean_text = [ wn.lemmatize(word, pos = "v") for word in word_tokenize(text.lower()) if black_txt(word)]
    clean_text2 = [word for word in clean_text if black_txt(word)]
    return " ".join(clean_text2)
#all_data["auhurs_genres"] = all_data["genres"].map(str) + "  " + all_data["authors"]
#all_data = all_data.drop(['genres', 'authors'], axis = 1)
all_data['genres'] = all_data['genres'].astype(str)
all_data['authors'] = all_data['authors'].astype(str)
all_data['title'] = all_data['title'].astype(str)
all_data['genres'] = all_data['genres'].apply(clean_txt)
all_data['authors'] = all_data['authors'].apply(clean_txt)
all_data['title'] = all_data['title'].apply(clean_txt)
print(all_data.shape)
all_data.head(3)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
'''with open('all_data', 'wb') as f:
    pickle.dump(all_data, f)'''

Заменим пропущенный рейтинг невозможное значение

In [ ]:
all_d = all_data.copy(deep = True)
all_d['rating'] = all_d['rating'].fillna(-1)
print(all_d.shape)
all_d.tail(5)

Тааак) пока не поздно выведем давай что будем оценивать. Функции оценки назовем и добавим этот признак. Добавим популярность книг в отдельный столбец. И думаю попробуем коэфицент для каждой последующей книги увеличивать для пользователя начнем с этого. Так же с нашей функцие те кто читают чаще получают больший коэфицент на последние книги прочитанные

In [ ]:
# Оценим книги от одного до двух каждым пользователем где каждая последующая книга с которой было взаимодействие оценивается больше предыдущего. 
# Даты у нас последовательно выстроены, поэтому не смотрим сортировку
list_id = all_d.user_id.unique().tolist()
next_book = {}
for i in list_id:
    time_list = all_d.loc[all_d.user_id == i]['start_date'].index
    ind = 0
    for j in time_list:
        next_book[j] = 1 + (ind/(len(time_list)))  
        ind += 1
all_d = all_d.join(pd.DataFrame(pd.Series(next_book)))
all_d = all_d.rename({0: 'book_rat'}, axis = 1)

# Ну и перестроим таблицу дат по количеству дней оставшихся до конечной десяти дней после последней записи нашей, ну по количеству прогнозируемых книг
all_d['days'] = (datetime.datetime(2020, 1, 10) - all_d['start_date']).dt.days
all_d['days'] = all_d['days'].fillna(-1)


# Добавим каждому пользователю количество прочитанных ими книг
user_book = pd.DataFrame(all_d.groupby('user_id').count()['item_id']).rename({'item_id': 'user_book'}, axis = 1).reset_index()
all_d = all_d.merge(user_book, how = 'left')

# Добавим количество взаимодействий с каждой книгой всеми пользователями
count_book = pd.DataFrame(all_d.groupby('item_id').count()['user_id']).rename({'user_id': 'count_book'}, axis = 1).reset_index()
all_d = all_d.merge(count_book, how = 'left')
all_d['count_book'] = all_d['count_book'].fillna(0)
all_d['count_book'] = all_d['count_book'].astype('int16')

print(all_d.shape)        
all_d.tail(5)

Приступим к кодированию категориальных признаков и исправим формат кодирования авторов

In [ ]:
all_data_code = all_d.copy(deep = True)
encod_age = LabelEncoder()
all_data_code.age = encod_age.fit_transform(all_data_code.age)

encod_sex = LabelEncoder()
all_data_code.sex.replace(0, 'f', inplace = True)
all_data_code.sex.replace(1, 'm', inplace = True)
all_data_code.sex = encod_sex.fit_transform(all_data_code.sex)

encod_year = LabelEncoder()
all_data_code.year = encod_year.fit_transform(all_data_code.year)

print(all_data_code.shape)
all_data_code.tail(3)

Ну теперь используем итеративную замену пропущенных значений на оставшихся данных и вернем текстовый столбец

In [ ]:
temp = all_data_code[['title', 'genres', 'authors', 'start_date']]
all_data_code = all_data_code.drop(['title', 'genres', 'authors', 'start_date'], axis = 1)
imp_mean = IterativeImputer(random_state = 0)
imp_mean.fit(all_data_code)
all_data_code = imp_mean.transform(all_data_code)
all_data_code = pd.DataFrame(all_data_code)
all_data_code.columns = all_d.drop(['title', 'genres', 'authors', 'start_date'], axis = 1).columns
all_data_code[['title', 'genres', 'authors', 'start_date']] = temp
print(all_data_code.shape)
all_data_code.tail(3)

In [ ]:
all_data_code.genres = all_data_code.genres.fillna('nogenres')
all_data_code.title = all_data_code.title.fillna('notitle')
all_data_code = all_data_code.drop(['authors_code'], axis = 1)
print(all_data_code.shape)
all_data_code.tail(3)

Теперь изменим целевой признак составив одну функцию из рейтинга прогресса и выведенных нами функций

In [ ]:
all_data_code['metric'] = (all_data_code['rating'] + 1).multiply(((all_data_code['progress'] / 100) + 1), axis = 'rows')
print(all_data_code.shape)
all_data_code.tail(5)

Перераспределим столбцы для удобства. чтоб потом легче было брать признаки книг отдельно и признаки пользователей, не добавляя 20ый и 21 года. их давно надо было убрать)) таблица взаимодействий их не включает) Решил упорядочить так: индексы, данные взаимодействий, данные пользователей, данные по книгам

In [ ]:
all_data_code = all_data_code[['user_id', 'item_id', 'rating', 'progress', 'days', 'age', 'sex', 'user_book',
       'year', 'count_book', 'title', 'genres', 'authors', 'start_date']]
all_data_code.user_id = all_data_code.user_id.astype(int)
all_data_code.item_id = all_data_code.item_id.astype(int)
all_data_code.tail(5)

**Контрольная точка** как в играх))) база воскрешения. Сохраню себе обработанную базу

In [ ]:
'''with open('all_data_mts', 'wb') as f:
    pickle.dump(all_data_code, f)'''
with open('/content/drive/MyDriv/MTS_summer_school/all_data_mts', 'rb') as f:
    all_data_code = pickle.load(f)
submission = pd.read_csv('/content/drive/MyDriv/MTS_summer_school/sample_submission.csv')

Начнем готвить данные сети нашей)

In [ ]:
df_all = all_data_code.sample(frac = 1).reset_index(drop = True)
print(df_all.shape)
df_all.tail(5)

Уберем часть данных касающиеся непопулярных книг и повторим чистку уже более глубокую после обучения но сохранив пользователей которые находятся в целевой группе. Ну и включим  в просчет записи которые на год старше прогнозируемой даты

In [ ]:
#temp = df_all.loc[(df_all.days > 720) & (df_all.days < 755)]
temp2 = df_all.loc[(df_all.days > 372) & (df_all.days < 382)]
temp3 = df_all.loc[(df_all.days < 100)]
df_all = pd.concat([temp2,temp3])
df_all = df_all.drop_duplicates(subset = ['item_id', 'user_id'])
print(df_all.shape)
df_all.tail(5)

In [ ]:
# Добавляем снова убранных пользователей входящих целевой список
list_temp = set(submission.Id) - set(df_all.user_id)
for i in list_temp:
    df_all = pd.concat([df_all, all_data_code.loc[all_data_code.user_id == i]])
    
print(df_all.item_id.shape)
set(submission.Id) - set(df_all.user_id)

Разбиваем данные для упаковки подчи в сеть

In [ ]:
df_all['item_next'] = df_all['item_id']
print(df_all.shape)

In [ ]:
list_user = df_all.drop_duplicates(subset = ['user_id'])['user_id'].values
len(list_user)

Сделаем колонку id книг со смещением в покупку для пользователя, чтоб заставить сеть предсказывыть id с эмбединга, ну и уберем данные без целевых данных

In [ ]:
for i in range(len(list_user)):
    df_all.loc[df_all.user_id == list_user[i], 'item_next'] = df_all.loc[df_all.user_id == list_user[i], 'item_next'].astype(int).shift(periods = -1)
    if i % 1000 == 0:
        print(i, end = "  -  ")
print(df_all.shape)
df_all.tail(5)

In [ ]:
print(df_all.shape)
#print(miss(df_all).head(5))
print('кол-во пользователей в датасете и количество пользователей которые остануться после удаления нулей', 
      df_all.drop_duplicates(subset = ['user_id']).shape[0], ' - ', df_all.dropna(inplace = False).drop_duplicates(subset = ['user_id']).shape[0])
print(df_all.shape)
df_all.tail(3)

**Контрольная точка** вторая))) база воскрешения. Сохраню себе обработанную базу

In [ ]:
with open('df_all', 'wb') as f:
    pickle.dump(df_all, f)
'''with open('./df_all', 'rb') as f:
    df_all = pickle.load(f)
submission = pd.read_csv('/content/drive/MyDriv/MTS_summer_school/sample_submission.csv')'''

Векторизируем)) кодируем данные. авторов к примеру горячим кодированием как текст ссмысла нет, а вот жанры в вектора

In [ ]:
max_len = 500
vectorizer_t = TfidfVectorizer(max_features = max_len)
vectorizer_g = TfidfVectorizer(max_features = max_len)

vectorizer_t.fit(df_all['title'])
vectorizer_g.fit(df_all['genres'])

encod_user_id = LabelEncoder()
encod_item_id = LabelEncoder()
encod_authors = LabelEncoder()

encod_user_id.fit(df_all.user_id)
encod_item_id.fit(np.hstack((df_all.item_id.values, df_all.item_next.values)))
encod_authors.fit(df_all.authors)

In [ ]:
data_shift = df_all.copy(deep = True)
print(data_shift.item_id.shape)
data_shift.dropna(inplace = True)
print(data_shift.item_id.shape)

Назначим отсутствующим значениям нелогичный индекс

In [ ]:
data_shift.item_next = data_shift.item_next.fillna(-1)
data_shift.item_next = data_shift.item_next.astype(int)
print(data_shift.shape)
data_shift.tail(3)

Глянем распределение прогресса

In [ ]:
group_labels = ['progress']
hist_data = [data_shift.progress.values.tolist()]
size = (max(data_shift.progress)-min(data_shift.progress))/30
color = ['#FF9988']
fig = ff.create_distplot(hist_data, group_labels,  histnorm = 'probability density', show_rug = False, bin_size = size, colors = color)
fig.show()

Применяем нормирование признаков BoxCox и посмотрим еще раз)

In [ ]:
'''boxcox_feature = df_all.metric
boxcox_feature, _ = stats.boxcox(boxcox_feature)
group_labels = ['metric']
hist_data = [boxcox_feature]
size = (max(boxcox_feature)-min(boxcox_feature))/30
color = ['#FF9988']
fig = ff.create_distplot(hist_data, group_labels,  histnorm = 'probability density', show_rug = False, bin_size = size, colors = color)
fig.show()
df_all.metric = boxcox_feature'''

Создание словарей сопоставления идентификаторов пользователей и книг для ограничения эмбединга количеством имеющимся. Не забыть главное после, вурнеть

In [ ]:
data_encode = data_shift.copy(deep = True)
data_encode = data_encode.reset_index().drop(['index'], axis = 1)
data_encode.user_id = encod_user_id.transform(data_encode.user_id)
data_encode.item_id = encod_item_id.transform(data_encode.item_id)
data_encode.item_next = encod_item_id.transform(data_encode.item_next)

data_encode.authors = encod_authors.transform(data_encode.authors)
data_encode.loc[data_encode.user_id == data_encode.iloc[0,1]]

Подготовим параметры под стандартизацию. Нужно стандартизировать параметры чтобы параметры с маленькими цифрами не игнорировались сетью. С нормированием Box Cox сеть отработала лучше

In [ ]:
'''scaler = StandardScaler()
scaler.fit(data_encode[['rating', 'progress', 'count_book', 'user_book', 'year', 'days']])'''

In [ ]:
data_encode.rating, _ = stats.boxcox(data_encode.rating + 1.1)
data_encode.progress, _ = stats.boxcox(data_encode.progress + 0.1)
#data_encode.days, _ = stats.boxcox(data_encode.days)
data_encode.count_book, _ = stats.boxcox(data_encode.count_book)
data_encode.user_book, _ = stats.boxcox(data_encode.user_book)
data_encode.year, _ = stats.boxcox(data_encode.year + 0.1)
print(data_encode.shape)
data_encode.tail(3)

Разобъем данные для проверки выделив последние два дня из имеющего набора

In [ ]:
all_train = data_encode.loc[data_encode.days >= 12]
all_test = data_encode.loc[data_encode.days < 12]
all_train.shape, all_test.shape

Токенизируем векторизируем отделяем))

In [ ]:
vec_title_train = vectorizer_t.transform(all_train['title']).toarray()
vec_genres_train = vectorizer_g.transform(all_train['genres']).toarray()
all_train = all_train.drop(['title', 'genres'], axis = 1)

vec_title_test = vectorizer_t.transform(all_test['title']).toarray()
vec_genres_test = vectorizer_g.transform(all_test['genres']).toarray()
all_test = all_test.drop(['title', 'genres'], axis = 1)

print(len(vec_title_train), len(vec_title_test))
all_train.tail(3)

Разбиваем файлы на группы для подачи в сеть предварительно применив стандартизацию

In [ ]:
#all_train[['rating', 'progress', 'count_book', 'user_book', 'year', 'days']] = scaler.transform(all_train[['rating', 'progress', 'count_book', 'user_book', 'year', 'days']])
df_user_train = all_train['user_id']
df_item_train = all_train['item_id']
df_next_train = all_train['item_next']
df_vec_title_train = vec_title_train
df_vec_genres_train = vec_genres_train
df_feature_train = all_train.drop(['user_id', 'item_id', 'start_date', 'item_next'], axis = 1)
print(df_user_train.shape, df_item_train.shape, df_next_train.shape, df_vec_title_train.shape, df_vec_genres_train.shape, df_feature_train.shape)
df_feature_train.head(3)

In [ ]:
#all_test[['rating', 'progress', 'count_book', 'user_book', 'year', 'days']] = scaler.transform(all_test[['rating', 'progress', 'count_book', 'user_book', 'year', 'days']])
df_user_test = all_test['user_id']
df_item_test = all_test['item_id']
df_next_test = all_test['item_next']
df_vec_title_test = vec_title_test
df_vec_genres_test = vec_genres_test
df_feature_test = all_test.drop(['user_id', 'item_id', 'start_date', 'item_next'], axis = 1)
print(df_user_test.shape, df_item_test.shape, df_next_test.shape, df_vec_title_test.shape, df_vec_genres_test.shape, df_feature_test.shape)
df_feature_test.head(3)

Пришло время разбить данные на тренировочные и тестовы и валидационные. почти все данные в тренировочных потому что я наэксперементировался уже)) и тут не такой большой риск переоучения думаю, ведь данные одни и те же и новые данные не поступают в сеть, только те на которых он обучится, только комбинируя две части(книги и пользователи)

In [ ]:
split = int(df_user_train.shape[0]*.85)

user_train = df_user_train[:split]
item_train = df_item_train[:split]
next_train = df_next_train[:split]
feature_train = df_feature_train[:split]
title_train = df_vec_title_train[:split]
genres_train = df_vec_genres_train[:split]

user_val = df_user_train[split:]
item_val = df_item_train[split:]
next_val = df_next_train[split:]
feature_val = df_feature_train[split:]
title_val = df_vec_title_train[split:]
genres_val = df_vec_genres_train[split:]

Определяем количество уникальных пользователей и книг

In [ ]:
user_count = len(encod_user_id.classes_)
item_count = len(encod_item_id.classes_)
print(user_count)
print(item_count)

Ну чтожжжж) запустим генераторы TensorFlow для подачи данных под TPU с ними мнооого проблем было у меня, информации мало, ограничений много в форматах и т.д. свежая часть библиотеки (ну ладно, сырая)))

In [ ]:
batch_size = 256 * strategy.num_replicas_in_sync
auto = tf.data.experimental.AUTOTUNE

tf_train = (
    tf.data.Dataset
    .from_tensor_slices(({"user": user_train, "item": item_train, "feature": feature_train, "title": title_train, "genres": genres_train}, next_train))
    .batch(batch_size).prefetch(auto)
)

tf_val = (
    tf.data.Dataset
    .from_tensor_slices(({"user": user_val, "item": item_val, "feature": feature_val, "title": title_val, "genres": genres_val}, next_val))
    .batch(batch_size).prefetch(auto)
)

tf_test = (
    tf.data.Dataset
    .from_tensor_slices(({"user": df_user_test, "item": df_item_test, "feature": df_feature_test, "title": df_vec_title_test, "genres": df_vec_genres_test}, df_next_test))
    .batch(batch_size).prefetch(auto)
)

Напишем лосс функцию для нашей задачки с маской в случае поподания нулевого значения

In [ ]:
def loss_function(real, pred):

    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_object_ = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True,
                                                                 reduction = 'none')
    loss_ = loss_object_(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

Так же нам надо определить в эту странную функцию саму модель) будем выносить вычисления в тензоры все дела))) Когда эта штука взлетает, настроение взлетает тоже. кстати могли жанрам проставить индексы и ввести так же как через эмбединг слой

In [ ]:
with strategy.scope():
    
    embedding_dimension  =  100

    layer_u_inp = Input(shape = [1], name = 'user')
    layer_i_inp = Input(shape = [1], name = 'item')
    layer_f_inp = Input(shape = [df_feature_train.shape[1]], name = 'feature')
    layer_g_inp = Input(shape = [genres_train.shape[1]], name = 'genres', sparse = True)
    layer_t_inp = Input(shape = [title_train.shape[1]], name = 'title', sparse = True)

    # Создаем вложение эмбедингов для пользователей и книг
    layer_u_emb = Embedding(output_dim = embedding_dimension,
                              input_dim = user_count,
                              input_length = 1,
                              name = 'user_embedding')(layer_u_inp)

    layer_i_emb = Embedding(output_dim = embedding_dimension,
                                input_dim = item_count,
                                input_length = 1,
                                name = 'item_embedding')(layer_i_inp) #, mask_zero=True

   
    layer_f_emb = Dense(512, activation = 'relu')(layer_f_inp)
    layer_f_emb = BatchNormalization()(layer_f_emb)
    layer_f_emb = Dense(512, activation = 'relu')(layer_f_emb)
    
    # Уменьшаем размерность метаданных
    layer_t_emb = Dense(512, activation = 'relu')(layer_t_inp)
    layer_t_emb = BatchNormalization()(layer_t_emb)
    layer_t_emb = Dense(512, activation = 'relu')(layer_t_emb)
    
    # Уменьшаем размерность метаданных
    layer_g_emb = Dense(512, activation = 'relu')(layer_g_inp)
    layer_g_emb = BatchNormalization()(layer_g_emb)
    layer_g_emb = Dense(512, activation = 'relu')(layer_g_emb)

    # Изменение формы встраиваемых слоев
    layer_u_reshape = Reshape([embedding_dimension])(layer_u_emb)
    layer_i_reshape = Reshape([embedding_dimension])(layer_i_emb)

    # Объеденяем слои
    layer_concate = Concatenate()([layer_u_reshape, layer_i_reshape, layer_f_emb, layer_t_emb, layer_g_emb])


    # Добавляем слои сети
    
    dense = Dense(2048, activation = 'relu')(layer_concate) #kernel_regularizer = regularizers.l2(0.0001)
    dense = BatchNormalization()(dense)
    dense = Dense(2048, activation = 'relu')(dense)
    dense = Dropout(0.4)(dense)
    dense = Dense(2048, activation = 'relu')(dense)
    dense = Dropout(0.4)(dense)

    output = Dense(item_count, activation = 'relu', name = 'output')(dense)
    
    opt = Adam(learning_rate = 0.00001)
    
    loss =  loss_function
    
    
    # Компиляция модели
    model = Model(inputs = [layer_u_inp, layer_i_inp, layer_f_inp, layer_t_inp, layer_g_inp], outputs = output)
    model.compile(loss = loss, optimizer = opt, metrics = ['sparse_categorical_accuracy'])

model.summary()

Пора обучать нашу модель))) Я ей горжусь) эпох много для такого задания, данных много, бачи большие и установлю стоп обучения если 10 итераций подряд не будет повышаться на валидации ошибка, оптимизатор думаю хорошо для этого подобран и скорость его обучения)) ползет как черепаха, не застревает и спускается к нужной точке медленно, выйти из ямы минимума не успеет из за функции остановки. я им доволен

In [ ]:
n_epochs = 5 # 5 / 15 / 

early_stop = EarlyStopping(monitor = 'val_sparse_categorical_accuracy', mode = 'max', verbose = 1, patience = 100)
history = model.fit(tf_train,
          batch_size = batch_size,
          epochs = n_epochs,
          validation_data = tf_val,
          verbose = 1, callbacks = [early_stop])

Запустим тестовые прогнозы) Что интересно нам скажет модель. Думале еще кросс валидацию написать, но на нашей задаче и это не так важно из за того что мы не так сильно боимся переобучиться. Кстати я долго копался, функция validation_split не работает пока в TPU. Данные надо подовать через спец генераторы. и в нашей задаче чтоб сделать кросс валидацию, надо писать функцию самим. Ну думаю это не сложно будет тому кому надо будет сделать

In [ ]:
y_pred = model.predict(tf_test)
y_true = df_next_test.values

predict = pd.DataFrame(y_pred * 10000)
predict['True'] = pd.DataFrame(y_true)
predict = predict.rename({0: 'Predict'}, axis=1)
predict.head(5)

глянем индексы с максимальными значениями

In [ ]:
np.argsort(y_pred[0])[-10:]

Подготовим данные под оценку метрики

In [ ]:
test_id_metric = []
for i in y_true:
    test_id_metric.append([i])
pred_id_metric = []
test_metric = []
for i in range(len(y_true)):    
    pred_id_metric.append(np.argsort(y_pred[i])[-50:])
    test_metric.append(np.argsort(y_pred[i])[-10:])

In [ ]:
ml_metrics.mapk(test_id_metric, test_metric, 10)

Обработаем предскаания, вместо десяти берем больше чтоб от них еще отобрать более популярные. Хотя моя сеть эти данные и так получает

In [ ]:
data = df_all.loc[(df_all.days < 50)]
test_data = all_data_code.loc[all_data_code.days >= 12]
predict_list = []
for i in range(len(pred_id_metric)):
    list_buy = test_data.loc[test_data.user_id ==  encod_user_id.inverse_transform(df_user_test.values)[i]]['item_id'].values
    dict_pred = {}
    list_pred = encod_item_id.inverse_transform(pred_id_metric[i]).astype(int)
    list_pred = list(set(list_pred) - set(list_buy))
    for j in list_pred:
        dict_pred[j] = data.loc[data.item_id == j].shape[0]
    dict_pred = {k: v for k, v in sorted(dict_pred.items(), key=lambda item: item[1], reverse=True)}
    list_pred = list(dict_pred.keys())
    predict_list.append(list_pred[:10])

true_list = []
id_true = encod_item_id.inverse_transform(y_true).astype(int)
for i in id_true:
    true_list.append([i])
true_list
len(predict_list), len(true_list)

In [ ]:
print(predict_list[0])
print(predict_list[1])
print(predict_list[2])
print(predict_list[3])
print(predict_list[4])

In [ ]:
ml_metrics.mapk(true_list, predict_list, 10)

тесты закончили, выведем предсказания уже на выборке для прогнозирования 

In [ ]:
pred_data_target = df_all.sort_values(by = 'days').drop_duplicates(subset = ['user_id'])
pred_data_target = pred_data_target.loc[pred_data_target.user_id.isin(submission.Id.values.tolist())]
pred_data_target.user_id = encod_user_id.transform(pred_data_target.user_id)
pred_data_target.item_id = encod_item_id.transform(pred_data_target.item_id)
print(pred_data_target.shape)
pred_data_target.head(3)

In [ ]:
temp = submission.copy(deep = True)
temp.Id = encod_user_id.transform(temp.Id)
temp = temp.rename({'Id': 'user_id'}, axis = 1)
pred_data_target = temp.drop(['Predicted'], axis = 1).merge(pred_data_target, how = 'left')
pred_data_target.authors = encod_authors.transform(pred_data_target.authors)
print(pred_data_target.shape)
pred_data_target.head(3)

In [ ]:
pred_data_target.rating, _ = stats.boxcox(pred_data_target.rating + 1.1)
pred_data_target.progress, _ = stats.boxcox(pred_data_target.progress + 0.1)
#data_encode.days, _ = stats.boxcox(data_encode.days)
pred_data_target.count_book, _ = stats.boxcox(pred_data_target.count_book + 0.1)
pred_data_target.user_book, _ = stats.boxcox(pred_data_target.user_book + 0.1)
pred_data_target.year, _ = stats.boxcox(pred_data_target.year + 0.1)

In [ ]:
pred_vec_title = vectorizer_t.transform(pred_data_target['title']).toarray()
pred_vec_genres = vectorizer_g.transform(pred_data_target['genres']).toarray()
pred_df_user = pred_data_target['user_id']
pred_df_item = pred_data_target['item_id']
pred_df_feature = pred_data_target.drop(['user_id', 'item_id', 'title', 'genres', 'item_next', 'start_date'], axis = 1)
print(pred_df_user.shape, pred_df_item.shape, pred_vec_title.shape, pred_vec_genres.shape, pred_df_feature.shape)

In [ ]:
tf_pred = (
    tf.data.Dataset
    .from_tensor_slices(({"user": pred_df_user, "item": pred_df_item, "feature": pred_df_feature, "title": pred_vec_title, "genres": pred_vec_genres}))
    .batch(batch_size)
)

In [ ]:
predictions = model.predict(tf_pred)

In [ ]:
pred_id = []
for i in range(len(predictions)):    
    pred_id.append(np.argsort(predictions[i])[-50:])
pred_id[:2]

In [ ]:
data = df_all.loc[(df_all.days < 30)]

predict_list = []
for i in range(len(pred_id)):
    dict_pred = {}
    list_pred = encod_item_id.inverse_transform(pred_id[i]).astype(int)
    for j in list_pred:
        dict_pred[j] = data.loc[data.item_id == j].shape[0]
    dict_pred = {k: v for k, v in sorted(dict_pred.items(), key=lambda item: item[1], reverse=True)}
    list_pred = list(dict_pred.keys())
    predict_list.append(list_pred[:10])


In [ ]:
predict_list[0]

In [ ]:
for i in range(len(predict_list)):
    submission.Predicted[i] = ' '.join(str(e) for e in predict_list[i])

Урааа)) готовы отправлять результаты)

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv', index = False)